# Имена и объекты

## Ячейки памяти

Память компьютера состоит из [ячеек](https://ru.wikipedia.org/wiki/Ячейка_памяти). Типичный размер ячейки — 8 битов, т.е. 1 байт.

У каждой ячейки есть ядрес. Для простоты мы можем считать, что нумерация одномерная и непрерывная, т.е. ячейки пронумерованы числами 0, 1, 2, 3 и т.д. (На самом деле адресация памяти устроена [сложнее](https://ru.wikipedia.org/wiki/Адресация_памяти). Непрерывная одномерная нумерация памяти — это удобная для нас **абстрация**, которая скрывает под собой множество деталей.)

## Адрес объекта

Все объекты, создающиеся при выполнении программы на Питоне, занимают место в памяти: несколько подряд идущих ячеек.

Адресом объекта считается адрес его первой ячейки.

Мы можем узнать адрес объекта с помощью встроенной функции [id](https://docs.python.org/3/library/functions.html#id).

In [1]:
s = "abc"
id(s)

140710286538560

## Равенство и тождество

**Равенство** объектов означает, что объекты **одинаковы**.

**Тождество** объектов означает, что перед нами **один объект под двумя разными именами**.

Равенство проверяется оператором `==`, тождество проверяется оператором `is`.

In [2]:
x = ("ab" + "c").upper()
y = "ABCD"[0:3]

print(x == y)
print(x is y)

True
False


In [3]:
z = y

print(y == z)
print(y is z)

True
True


In [4]:
print(id(x))
print(id(y))
print(id(z))

1779321337296
1779343692816
1779343692816


## Указатели

**Указатель** (pointer) — это объект, который хранит адрес другого объекта. Можно считать, что указатель — это целое положительное число плюс информация о том, что это число должно интерпретироваться как адрес.

Языки системного программирования (`C`, `C++`, `Rust` и т.д.) позволяют манипулировать указателями напрямую: например, перебирать объекты в диапазоне адресов. Питон такой возможности не даёт. Тем не менее, знать про указатели необходимо для понимания происходящего.

## Коллекции (list, set, dict)

На первых лекциях мы говорили о коллекциях как о контейнерах (коробках, ящиках), в которых лежат объекты. В первом приближении это полезная метафора, но сейчас она начинает нам мешать.

Заметим, что один и тот же объект может лежать в нескольких коллекциях.

Мы можем проверить, что это один и тот же объект с помощью оператора `is` или простым сравнением адресов.

In [5]:
obj = "example"

my_list = [obj, obj]
my_set = {obj}
my_dict = {"key": obj}

In [6]:
(
    obj
    is my_list[0]
    is my_list[1]
    is next(iter(my_set))
    is my_dict["key"]
)

True

In [7]:
print(id(obj))
print(id(my_list[0]))
print(id(my_list[1]))
print(id(next(iter(my_set))))
print(id(my_dict["key"]))

1779343693296
1779343693296
1779343693296
1779343693296
1779343693296


На самом деле в коллекциях хранятся **указатели** на объекты.

### Рекурсивные структуры данных

Метафора "коллекция = коробка" окончательно ломается, когда мы помещаем внутрь коллекции ссылку на неё же.

In [8]:
my_list = [0, 1, 2]
my_list.append(my_list)

print(my_list)
print(my_list[3])

[0, 1, 2, [...]]
[0, 1, 2, [...]]


In [9]:
my_list is my_list[3]

True

Бывают и взаимно-рекурсивные (вложенные друг в друга) структуры.

In [10]:
husband = {"name": "John", "wife": None}
wife = {"name": "Mary", "husband": None}

husband["wife"] = wife
wife["husband"] = husband

family = {"husband": husband, "wife": wife}

In [11]:
# Используем pretty-print, чтобы лучше разглядеть.
from pprint import pprint

pprint(family)

{'husband': {'name': 'John',
             'wife': {'husband': <Recursion on dict with id=1779339196736>,
                      'name': 'Mary'}},
 'wife': {'husband': {'name': 'John',
                      'wife': <Recursion on dict with id=1779339189376>},
          'name': 'Mary'}}


In [12]:
(
    family["wife"]
    is family["husband"]["wife"]
    is family["wife"]["husband"]["wife"]
    is family["husband"]["wife"]["husband"]["wife"]
    is family["wife"]["husband"]["wife"]["husband"]["wife"]
)

True

Лингвистический аналог прохода по закольцованным ссылкам: _муж своей жены_, _слуга своего господина_.

## Классы и экземпляры классов

Любой класс — это тоже объект: он хранится в памяти, имеет адрес, на него могут указывать указатели, он может содержать указатели и т.д.

In [13]:
class MyClass:
    "Пример класса"

    def my_method(self):
        "Пример метода"
        return

# Другое имя
MyClassAlias = MyClass

In [14]:
print(id(MyClass))
print(id(MyClassAlias))

1779322438144
1779322438144


In [15]:
MyClassAlias is MyClass

True

Экземпляры классов иногда называют объектами. Это неточное словоупотребление: правильнее называть их экземплярами (instance). Объектами являются и экземпляры, и сами классы, и методы классов. (**Everything is an object.**)

Мы можем использовать любое из имён класса для создания экземпляров.

In [16]:
obj1 = MyClass()
obj2 = MyClassAlias()

**Каждый экземпляр содержит в себе указатель на класс**: специальный атрибут `__class__`.

In [17]:
print(obj1.__class__)
print(obj2.__class__)

<class '__main__.MyClass'>
<class '__main__.MyClass'>


In [18]:
obj1.__class__ is obj2.__class__ is MyClass

True

В Питоне есть встроенная функция `type`, которая возвращает класс своего аргумента.

In [19]:
print(type(obj1))

<class '__main__.MyClass'>


Функции и методы тоже являются объектами, и у них есть свой класс.

In [20]:
print(MyClass.my_method.__class__)
print(obj1.my_method.__class__)

<class 'function'>
<class 'method'>


Всё в Питоне является объектами (включая классы / типы данных). Как далеко мы уйдём, проверяя классы классов?

Класс всех классов называется "тип". Он является и своим же собственным классом.

In [21]:
print(MyClass.__class__)

<class 'type'>


In [22]:
T = MyClass.__class__
T is T.__class__ is T.__class__.__class__

True

## Пространства имён

**Пространство имён** — это словарь, где ключи — это указатели на имена, а значения — это указатели на объекты, названные этими именами.

Своё пространство имён есть в каждом контексте: например, в файле вне функций, внутри функций в процессе выполнения и т.д.

Выражение типа `name = value` называется **присваивание** (присваивание значения имени в текущем пространстве имён).

**Переменная** — это пара "имя + значение". Само по себе имя не является переменной. Само по себе значение тоже не является переменной.

Заметим, что переменные слева и справа от оператора `=` используются по-разному: от левой переменной берётся только имя, а от правой — только значение.

## Параметры и аргументы функций

Рассмотрим пример:

In [23]:
def add(x, y):
    return x + y

a = 1
b = 2
c = add(a, b)

В определении функции `add` есть два **параметра**: `x` и `y`. Параметры — это имена без значений.

При вызове функции `add` туда передаются два **аргумента**: `1` и `2`. Аргументы — это значения без имён.

В момент вызова происходит связывание параметров с аргументами: имя `x` получает значение `1`, имя `y` получает значение `2`.

Заметим, что переданные в функцию объекты имеют по два имени: во внутреннем контексте это имена "x" и "y", а во внешнем контексте это имена "a" и "b".

## Сборщик мусора и время жизни объектов

Код, который мы пишем на Питоне, может только создавать объекты, но не уничтожать их.

In [24]:
# создаётся объект
# расходуется память
obj = MyClass()

Уничтожением объектов в фоновом режиме занимается сборщик мусора (garbage collector).

У каждого объекта есть счётчик входящих ссылок. Когда у объекта появляется новое имя — счётчик увеличивается на 1. Имя удаляется — счётчик уменьшается на 1. Ссылка на объект помещается в коллекцию или в поле класса — счётчик увеличивается на 1. И так далее.

Когда счётчик входящих ссылок показывает 0, сборщик мусора *может* удалить этот объект. Он не обязан делать это немедленно.

Неудалённые объекты без входящих ссылок некоторое время существуют *где-то*. Они недостижимы для работающей программы, т.к. ссылок, ведущих к ним, не осталось.

([Подъём зомби-объектов](https://en.wikipedia.org/wiki/Object_resurrection) возможен в исключительных обстоятельствах.)

Вывод: **объект гарантированно жив, пока кто-нибудь про него помнит**.

Оператор `del` удаляет имя из текущего пространства имён, но не удаляет сам объект.

In [25]:
class Person:
    "Человек: имя и список друзей."

    def __init__(self, name):
        self.name = name
        self.friends = []

    def add_friend(self, person):
        self.friends.append(person)

    def __str__(self):
        names = [f.name for f in self.friends]
        return f"Person(name = {self.name}, friends = {names})"

In [26]:
john = Person("John")
bill = Person("Bill")

john.add_friend(bill)

In [27]:
print(john)
print(bill)

Person(name = John, friends = ['Bill'])
Person(name = Bill, friends = [])


In [28]:
# Удаляем имя 'bill'.
del bill

In [29]:
# Билл жив, потому что про него помнит Джон.
print(john)
print(john.friends[0])

Person(name = John, friends = ['Bill'])
Person(name = Bill, friends = [])


In [30]:
print(bill)

NameError: name 'bill' is not defined